In [1]:
import pandas as pd
import numpy as np

In [8]:

def station_location():
    # get the Station, Unit, Control Area mapping
    mapping = pd.read_csv('./Dataset/Remote-Booth-Station.csv',header = 0,names = ['UNIT','C/A','Station','Line Name','Division'])

    # download dataset with longitude and altitude of station
    path = 'http://web.mta.info/developers/data/nyct/subway/Stations.csv'
    col_name_new = ['Station ID','Complex ID','GTFS Stop ID','Division','Line','Stop Name','Borough','Daytime Routes','Structure','GTFS Latitude','GTFS Longitude']
    location = pd.read_csv(path, sep=",", header=0, names = col_name_new)

    subset_mapping = mapping[["Division", "Station", "Line Name"]]
    subset_mapping["Station"] = subset_mapping["Station"].str.replace(" ", "").str.lower()
    subset_mapping["Line Name"] = subset_mapping["Line Name"].str.replace(" ", "").str.lower()

    subset_location = location[["Division","Stop Name", "Daytime Routes"]]
    subset_location["Stop Name"] = subset_location["Stop Name"].str.replace(" ", "").str.lower()
    subset_location["Daytime Routes"] = subset_location["Daytime Routes"].str.replace(" ", "").str.lower()

    df = pd.DataFrame()
    from collections import Counter
    for i in subset_mapping.index:
        sub_each = subset_location.loc[(subset_location["Division"] == subset_mapping.loc[i,"Division"]) \
                                             #& ((subset_location["Daytime Routes"] == subset_mapping.loc[i,"Line Name"]) | (subset_location["Stop Name"] == subset_mapping.loc[i,"Station"])) \
                                             , ["Stop Name","Daytime Routes"]]
        sub_each_final = sub_each.copy()

        # if find nothing, fill with NaN
        if len(sub_each) == 0:
            sub_each_final = pd.Series(np.nan, index = [np.nan])

        # Find the most common charaters in "Station" and "Line Name" matching the mapping csv
        if len(sub_each) > 1:
            sub_each_final = pd.Series()
            final = 0
            for j in range(len(sub_each.values)):
                string1 = sub_each.values[j][0]
                string2 = subset_mapping.loc[i,"Station"]
                C1 = Counter(string1)&Counter(string2)
                temp1 = sum(C1.values())/sum(Counter(string2).values())
                string3 = sub_each.values[j][1]
                string4 = subset_mapping.loc[i,"Line Name"]
                C2 = Counter(string3)&Counter(string4)
                temp2 = sum(C2.values())/sum(Counter(string4).values())
                temp = temp1+temp2
                final = max(final, temp)
                if temp == final:
                    sub_each_final = pd.Series(sub_each.values[j][0], index = [sub_each.index.values[j]])
        df["index"+str(i)] = sub_each_final.index.values
        df["sub_each" + str(i)] = sub_each_final.values
        #df.set_value(index = [0:len(sub_each)],col = "index"+str(i), value = sub_each.index.values)
        #df.set_value(index = [0:len(sub_each)], col = "sub_each" + str(i), value = sub_each.to_string(index=False))


    mapping['Station_modified'] = df.iloc[0,1::2].values
    location_new = location.copy()
    location_new["Station_modified"] = location_new["Stop Name"].str.replace(" ", "").str.lower()
    location_final = location_new.loc[:,['Station ID','GTFS Latitude', 'GTFS Longitude', 'Station_modified']]

    mapping_final = pd.merge(mapping, location_final, how='left', on=['Station_modified'])
    
    return mapping_final

In [9]:
mapping_final = station_location()

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [10]:
mapping_final

,UNIT,C/A,Station,Line Name,Division,Station_modified,Station ID,GTFS Latitude,GTFS Longitude
0,R001,A060,WHITEHALL ST,R1,BMT,whitehallst,23.0,40.703087,-74.012994
1,R001,A058,WHITEHALL ST,R1,BMT,whitehallst,23.0,40.703087,-74.012994
2,R001,R101S,SOUTH FERRY,R1,IRT,southferry,330.0,40.702068,-74.013664
3,R002,A077,FULTON ST,ACJZ2345,BMT,fultonst,106.0,40.710374,-74.007582
4,R002,A077,FULTON ST,ACJZ2345,BMT,fultonst,172.0,40.710197,-74.007691
5,R002,A077,FULTON ST,ACJZ2345,BMT,fultonst,292.0,40.687119,-73.975375
6,R002,A077,FULTON ST,ACJZ2345,BMT,fultonst,332.0,40.709416,-74.006571
7,R002,A077,FULTON ST,ACJZ2345,BMT,fultonst,412.0,40.710368,-74.009509
8,R002,A081,FULTON ST,ACJZ2345,BMT,fultonst,106.0,40.710374,-74.007582
9,R002,A081,FULTON ST,ACJZ2345,BMT,fultonst,172.0,40.710197,-74.007691


In [13]:
mapping_final.to_csv(path_or_buf='./mapping_final.csv',sep=',', index=None)